In [62]:
%%bash
pip install idx2numpy
pip install jupyterthemes

  Created wheel for idx2numpy: filename=idx2numpy-1.2.3-py3-none-any.whl size=7904 sha256=7f396bc6fadbaa22a4c0e68cb8197fb32df28d9fad583be39088bea472b8e18d
  Stored in directory: /home/ec2-user/.cache/pip/wheels/e7/34/28/e6938b3277d94e90acb425f0ea451fd3d4be7713e13f19966f
Successfully built idx2numpy


In [48]:
# create a darker background template (better for my eyes)
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('chesterish')

In [150]:
# console and directory access
import os
import re
import shutil
import datetime

# interacting with Amazon AWS
import boto3
from sagemaker.session import Session

# data reading and exporting  
import json
import tempfile
import pandas as pd
import numpy as np
import struct

# parsing SEC website for data  
import requests
import time 
import idx2numpy
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

In [4]:
file_type = 'X-17A-5'       # files looking to extract
prior2date = '20201231'     # format YYYY/MM/DD - select data prior to this data

In [5]:
bucket = "ran-s3-systemic-risk"
data_folder ="Input/X-17A-5/"

# Amazon Textract client and Sagemaker session
s3 = boto3.client('s3')
session = Session()

In [6]:
os.getcwd()

'/home/ec2-user/SageMaker/SEC_X17A5'

## Dealer Data Import
Parses in dealer information with accompaning CIK code for EDGAR lookup - data is taken from SEC dealer registration

In [7]:
if os.path.isfile('CIKandDealers.txt'):
    print('File found')
    
    # exporting RegisteredDealer information, loading in JSON dictionary 
    with open('CIKandDealers.txt') as file:
        cik2brokers = json.load(file)
    
    # unpacking the dictionary keys (all broker dealer CIK figures)
    bdNames = [*cik2brokers]
    
else:
    print('File not found, retrieving information ...')
    start = time.time()
    
    # will be used to concat all available broker dealer information 
    tempDF = []
    
    # send request to SEC website to retrieve broker dealer information 
    response = requests.get('https://www.sec.gov/help/foiadocsbdfoiahtm.html', allow_redirects=True)

    # parse the HTML doc string from the response object
    s1Table = BeautifulSoup(response.text, 'html.parser') 
    
    # parse through links from the SEC filings
    for link in s1Table.find_all('a'):
        documentURL = link.get('href')           # document links for filings
        
        try:
            # find .txt file substring greater than 0
            if documentURL.find('.txt') > 0:
                
                # requesting data from document links storing the files
                pdf_storage = requests.get('https://www.sec.gov'+ documentURL, allow_redirects=True)
                
                # open a file to store files from SEC
                open('secDealers.txt', 'wb').write(pdf_storage.content)
                
                # convert text file to .csv and store dataframe
                df = pd.read_csv('secDealers.txt', sep="\t", header=None)
                
                dateRelease = documentURL.split('/')[-1][2:-4]
                print('\tSEC Dealer information has been downloaded ' + dateRelease)
                
                # append new dataframe figures to be concated
                tempDF.append(df)
                
        except AttributeError:
            pass
    
    # concat all disjoint dataframes
    df = pd.concat(tempDF)
    
    # remove NaN columns and provide column naming conventions
    df = df[df.columns[:-1]]
    df.columns = ['CIK NUMBER', 'COMPANY NAME', 'REPORTING FILE NUMBER', 'ADDRESS1',
                  'ADDRESS2', 'CITY', 'STATE CODE', 'ZIP CODE']
    
    # remove duplicate rows from the dataset
    df = df.drop_duplicates(subset='CIK NUMBER')
    df.to_csv('secRegisteredDealers.csv')

    # convert CIK and Company Name to dictionary
    cik2brokers = df[df.columns[:2]].set_index('CIK NUMBER').to_dict(orient='index')

    # unpacking the dictionary keys (all broker dealer CIK figures)
    bdNames = [*cik2brokers]

    # exporting RegisteredDealer information
    with open('CIKandDealers.txt', 'w') as file:
        json.dump(cik2brokers, file)
        file.close()
        
    print('Time taken is {} seconds'.format(time.time()-start))

File found


In [171]:
# HTTPError: HTTP Error 403: Forbidden
# additional archived data pre-2007 is stored at https://www.sec.gov/Archives/edgar/full-index/
baseURL = 'https://www.sec.gov/Archives/edgar/full-index/'
years = np.arange(1993, datetime.datetime.today().year+1)
quarters = ['QTR1', 'QTR2', 'QTR3', 'QTR4']

# itterate through years and quarters
for year in years[:1]:
    for qt in quarters[:1]:
        searchURL = '{}{}/{}/form.idx'.format(baseURL, year, qt)
        
        try:
            # send request to SEC website to retrieve broker dealer information 
            response = requests.get(searchURL, allow_redirects=True)
            s1table = BeautifulSoup(response.text, 'html.parser')
            
            with open('main.txt', 'w') as i:
                i.write(response.text)
        except:
            pass

In [180]:
pd.read_fwf('main.txt')

,Description: Master Index of EDGAR Dissemination Feed by Form Type
0,"Last Data Received: March 31, 1993"
1,Comments: webmaster@sec.gov
2,Anonymous FTP: ftp://ftp.sec.gov/edgar/
3,Form Type Company Name ...
4,----------------------------------------------...
5,13F-HR SMITH THOMAS W ...
6,13FCONP RBS PARTNERS L P /CT ...
7,CERTNYS STORAGE TECHNOLOGY CORP ...
8,NSAR-B MERRILL LYNCH LIFE VARIABLE ANNUIT...


In [179]:
pd.read_csv('main.txt', header=[1, 2, 3, 4, 5])

IndexError: list index out of range

In [8]:
# sample of CIK brokers & dealer retained from SEC directory 
pd.DataFrame.from_dict(cik2brokers, orient='index').reset_index().sort_values(by='COMPANY NAME').head()

,index,COMPANY NAME
4673,1647385,16 POINTS LLC
6466,80869,1717 CAPITAL MANAGEMENT COMPANY
3386,1421259,"1792 SECURITIES, LLC"
4924,1721381,"180 DEGREE CAPITAL BD, LLC"
3903,1482939,"1851 SECURITIES, INC"


## PDF File Extraction
Extract URL links per company filing to download accompaning X-17A-5 files from SEC EDGAR site

In [9]:
n = len(bdNames)                # number of broker/dealers
secURLS = [0]*n                 # initialize memory 
startTime = time.time()         # set tuner to track code 
completeYear = None             # initialize year to check for updates

secFormat = 'https://www.sec.gov/cgi-bin/browse-edgar?'     # SEC base url
dataSelect = 'action=getcompany&CIK={}&type={}&dateb={}'    # select params.
    
# iterate through the CIK's from the broker dealers to build lookup URLs
for name in range(n):       
    url = secFormat + dataSelect.format(bdNames[name], file_type, prior2date)
    secURLS[name] = url                     # storing the search URLs

In [10]:
secURLS[:5]

['https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1904&type=X-17A-5&dateb=20201231',
 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=2303&type=X-17A-5&dateb=20201231',
 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=2554&type=X-17A-5&dateb=20201231',
 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=3475&type=X-17A-5&dateb=20201231',
 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=3683&type=X-17A-5&dateb=20201231']

In [11]:
def secParse(completeYear:str, nLinks:int, secURLS:list, cik2brokers:dict, bdNames:list, file_type:str, 
             bucket:str, subFolder:str=None):
    """
    Parses SEC website for X-17A-5 files (void function)
    :param nLinks: (int) the number of links to extract from
    :return: None
    """
    # initialize time for process to run 
    startTime = time.time()
    
    # discover all of the pdfs that you want to parse
    s3_path = session.list_s3_files(bucket, subFolder)
    
    # the URL links for each SEC company
    for url_link in range(nLinks):

        # company name for broker dealer being downloaded
        companyName = cik2brokers[bdNames[url_link]]['COMPANY NAME']

        # logging info for when files are being downloaded
        print('{} - Downloading {} files for {} - CIK ({})'.format(url_link, file_type, 
                                                                   companyName, bdNames[url_link]))

        # requesting HTML data link from the EDGAR search results 
        response = requests.get(secURLS[url_link], allow_redirects=True)

        # parse the HTML doc string from the response object
        s1Table = BeautifulSoup(response.text, 'html.parser') 

        # parse the html-doc string for all instance of <a href=>
        for link in s1Table.find_all('a'):
            documentURL = link.get('href')           # document links for filings

            try:
                # Check for Archives header as those are contained in the filings
                check = documentURL.split('/')[1]    
                if check == 'Archives':                 

                    # document URL link for each SEC filing for given year 
                    pdf_url = 'https://www.sec.gov' + documentURL
                    year = documentURL.split('-')[1] 
                    
                    # data is organized linearly, by most recent issue first
                    # requesting data from document links storing the files
                    pdf_storage = requests.get(pdf_url, allow_redirects=True)

                    # table from filing detail Edgar table 
                    s2Table = BeautifulSoup(pdf_storage.text, 'html.parser') 

                    # extracts all link within the filing table, filtering for pdfs
                    filing_detail = s2Table.find_all('a')
                    extract_link = [file.get('href') for file in filing_detail]

                    # filter for all pdf links from the extracted file links  
                    pdf_files = [string for string in extract_link if 'pdf' in string]
                    pdf_file = 'https://www.sec.gov' + pdf_files[-1] 

                    # storing pdf details within the folder
                    pdf_name = subFolder + str(bdNames[url_link]) + '-' + year + '.pdf'

                    # if pdf file is stored locally avoid running script further
                    # simply continue to other company (we assume all present)
                    if pdf_name in s3_path: 
                        print('\tAll files for {} are downloaded'.format(companyName))
                        break

                    else:
                        # request the specific pdf file from the the SEC
                        pdf_storage = requests.get(pdf_file, allow_redirects=True)

                        # filename for the pdf to be stored in s3
                        fileName = str(bdNames[url_link]) + '-' +  year + '.pdf'

                        # save PDF contents to local file location 
                        open(fileName, 'wb').write(pdf_storage.content)

                        # save contents to AWS S3 bucket
                        with open(fileName, 'rb') as data:
                            s3.upload_fileobj(data, bucket, subFolder + fileName)

                        # remove local file after it has been created
                        os.remove(fileName)

                        print('\tSaved {} files for {} year {}'.format(file_type, companyName, year))
                        completeYear = year 

            # if documentURL has no split greater than length of 1, false link       
            except IndexError:
                pass

    print('Time taken in seconds is {}'.format(time.time() - startTime))

In [12]:
# call function to parse data from the SEC -> port to s3
secParse(prior2date[2:4], n, secURLS, cik2brokers, bdNames, file_type, bucket, data_folder)

0 - Downloading X-17A-5 files for ABRAHAM SECURITIES CORPORATION - CIK (1904)
	All files for ABRAHAM SECURITIES CORPORATION are downloaded
1 - Downloading X-17A-5 files for PROSPERA FINANCIAL SERVICES, INC. - CIK (2303)
	All files for PROSPERA FINANCIAL SERVICES, INC. are downloaded
2 - Downloading X-17A-5 files for AEI SECURITIES, INC. - CIK (2554)
	All files for AEI SECURITIES, INC. are downloaded
3 - Downloading X-17A-5 files for ALEXANDER INVESTMENT SERVICES CO. - CIK (3475)
	All files for ALEXANDER INVESTMENT SERVICES CO. are downloaded
4 - Downloading X-17A-5 files for ALLEN & COMPANY LLC - CIK (3683)
	All files for ALLEN & COMPANY LLC are downloaded
5 - Downloading X-17A-5 files for M. E. ALLISON & CO., INC. - CIK (3995)
	All files for M. E. ALLISON & CO., INC. are downloaded
6 - Downloading X-17A-5 files for ALLISON-WILLIAMS COMPANY - CIK (3999)
	All files for ALLISON-WILLIAMS COMPANY are downloaded
7 - Downloading X-17A-5 files for AMERICAN FIDELITY SECURITIES, INC. - CIK (500

	All files for DAVENPORT & COMPANY LLC are downloaded
62 - Downloading X-17A-5 files for SHARE FINANCIAL SERVICES, INC. - CIK (27148)
	All files for SHARE FINANCIAL SERVICES, INC. are downloaded
63 - Downloading X-17A-5 files for D.A. DAVIDSON & CO. - CIK (27182)
	All files for D.A. DAVIDSON & CO. are downloaded
64 - Downloading X-17A-5 files for C. R. DAVIS & COMPANY - CIK (27224)
	All files for C. R. DAVIS & COMPANY are downloaded
65 - Downloading X-17A-5 files for H. C. DENISON CO. - CIK (28088)
	All files for H. C. DENISON CO. are downloaded
66 - Downloading X-17A-5 files for DEVELOPMENT CORPORATION FOR ISRAEL - CIK (28471)
	All files for DEVELOPMENT CORPORATION FOR ISRAEL are downloaded
67 - Downloading X-17A-5 files for DIVERSIFIED SECURITIES, INCORPORATED - CIK (29269)
	All files for DIVERSIFIED SECURITIES, INCORPORATED are downloaded
68 - Downloading X-17A-5 files for DOFT & CO., INC. - CIK (29476)
	All files for DOFT & CO., INC. are downloaded
69 - Downloading X-17A-5 files fo

	All files for EDWARD D. JONES & CO., L.P. are downloaded
124 - Downloading X-17A-5 files for KEEFE, BRUYETTE & WOODS, INC. - CIK (54930)
	All files for KEEFE, BRUYETTE & WOODS, INC. are downloaded
125 - Downloading X-17A-5 files for PLANNERS FINANCIAL SERVICES, INC. - CIK (55028)
	All files for PLANNERS FINANCIAL SERVICES, INC. are downloaded
126 - Downloading X-17A-5 files for KIMELMAN & BAIRD, LLC - CIK (55798)
	All files for KIMELMAN & BAIRD, LLC are downloaded
127 - Downloading X-17A-5 files for C. L. KING & ASSOCIATES, INC. - CIK (55847)
	All files for C. L. KING & ASSOCIATES, INC. are downloaded
128 - Downloading X-17A-5 files for KMS FINANCIAL SERVICES, INC. - CIK (56355)
	All files for KMS FINANCIAL SERVICES, INC. are downloaded
129 - Downloading X-17A-5 files for KRAMBO CORPORATION - CIK (56752)
	All files for KRAMBO CORPORATION are downloaded
130 - Downloading X-17A-5 files for KUYKENDALL & SCHNEIDER, INC. - CIK (57052)
	All files for KUYKENDALL & SCHNEIDER, INC. are downloa

	All files for LPL FINANCIAL LLC are downloaded
184 - Downloading X-17A-5 files for PRUCO SECURITIES, LLC. - CIK (80887)
	All files for PRUCO SECURITIES, LLC. are downloaded
185 - Downloading X-17A-5 files for PUTNAM RETAIL MANAGEMENT LIMITED PARTNERSHIP - CIK (81254)
	All files for PUTNAM RETAIL MANAGEMENT LIMITED PARTNERSHIP are downloaded
186 - Downloading X-17A-5 files for QUINCY CASS ASSOCIATES, INCORPORATED - CIK (81546)
	All files for QUINCY CASS ASSOCIATES, INCORPORATED are downloaded
187 - Downloading X-17A-5 files for A.G. QUINTAL INVESTMENT COMPANY INC. - CIK (81567)
	All files for A.G. QUINTAL INVESTMENT COMPANY INC. are downloaded
188 - Downloading X-17A-5 files for SAMUEL A. RAMIREZ & COMPANY, INC. - CIK (81909)
	All files for SAMUEL A. RAMIREZ & COMPANY, INC. are downloaded
189 - Downloading X-17A-5 files for AON SECURITIES LLC - CIK (82282)
	All files for AON SECURITIES LLC are downloaded
190 - Downloading X-17A-5 files for VANDERBILT SECURITIES, LLC - CIK (83809)
	All 

	All files for MM GLOBAL SECURITIES, INC. are downloaded
244 - Downloading X-17A-5 files for UBS FINANCIAL SERVICES INC. - CIK (200565)
	All files for UBS FINANCIAL SERVICES INC. are downloaded
245 - Downloading X-17A-5 files for E. E. POWELL & COMPANY INC. - CIK (200597)
	All files for E. E. POWELL & COMPANY INC. are downloaded
246 - Downloading X-17A-5 files for ROMANO BROTHERS AND COMPANY - CIK (200648)
	All files for ROMANO BROTHERS AND COMPANY are downloaded
247 - Downloading X-17A-5 files for ST. GERMAIN SECURITIES, INC. - CIK (200664)
	All files for ST. GERMAIN SECURITIES, INC. are downloaded
248 - Downloading X-17A-5 files for SECURITY DISTRIBUTORS, LLC - CIK (200684)
	All files for SECURITY DISTRIBUTORS, LLC are downloaded
249 - Downloading X-17A-5 files for SMITH, MOORE & CO. - CIK (200724)
	All files for SMITH, MOORE & CO. are downloaded
250 - Downloading X-17A-5 files for ZIV INVESTMENT COMPANY - CIK (200854)
	All files for ZIV INVESTMENT COMPANY are downloaded
251 - Downlo

...

	Saved X-17A-5 files for KEEFE VENTURES, LLC year 02
7740 - Downloading X-17A-5 files for SAMUELS LANE RORY - CIK (1126174)
7741 - Downloading X-17A-5 files for WOL CORP - CIK (1129969)
7742 - Downloading X-17A-5 files for MCTIGUE TRADING LLC - CIK (1137967)
7743 - Downloading X-17A-5 files for R-G INVESTMENTS CORPORATION - CIK (1141349)
	Saved X-17A-5 files for R-G INVESTMENTS CORPORATION year 07
	Saved X-17A-5 files for R-G INVESTMENTS CORPORATION year 06
	Saved X-17A-5 files for R-G INVESTMENTS CORPORATION year 05
	Saved X-17A-5 files for R-G INVESTMENTS CORPORATION year 04
	Saved X-17A-5 files for R-G INVESTMENTS CORPORATION year 03
7744 - Downloading X-17A-5 files for R&R PROPRIETARY LLC - CIK (1146120)
7745 - Downloading X-17A-5 files for GENEVA CAPITAL INVESTMENTS, LLC - CIK (1146131)
7746 - Downloading X-17A-5 files for SAINT SOTER CAPITAL, LLC - CIK (1146241)
	Saved X-17A-5 files for SAINT SOTER CAPITAL, LLC year 06
	Saved X-17A-5 files for SAINT SOTER CAPITAL, LLC year 05
	Sa

	Saved X-17A-5 files for INTEGRAL SECURITIES, INC. year 05
	Saved X-17A-5 files for INTEGRAL SECURITIES, INC. year 03
	Saved X-17A-5 files for INTEGRAL SECURITIES, INC. year 02
7784 - Downloading X-17A-5 files for RELIANT TRADING LLC - CIK (910748)
	Saved X-17A-5 files for RELIANT TRADING LLC year 07
	Saved X-17A-5 files for RELIANT TRADING LLC year 06
	Saved X-17A-5 files for RELIANT TRADING LLC year 06
	Saved X-17A-5 files for RELIANT TRADING LLC year 05
	Saved X-17A-5 files for RELIANT TRADING LLC year 04
	Saved X-17A-5 files for RELIANT TRADING LLC year 03
	Saved X-17A-5 files for RELIANT TRADING LLC year 02
7785 - Downloading X-17A-5 files for UNITED HERITAGE FINANCIAL SERVICES, INC. - CIK (914892)
	Saved X-17A-5 files for UNITED HERITAGE FINANCIAL SERVICES, INC. year 07
	Saved X-17A-5 files for UNITED HERITAGE FINANCIAL SERVICES, INC. year 06
	Saved X-17A-5 files for UNITED HERITAGE FINANCIAL SERVICES, INC. year 05
	Saved X-17A-5 files for UNITED HERITAGE FINANCIAL SERVICES, INC.

	Saved X-17A-5 files for HORATIO CAPITAL LLC year 07
	Saved X-17A-5 files for HORATIO CAPITAL LLC year 06
	Saved X-17A-5 files for HORATIO CAPITAL LLC year 05
7815 - Downloading X-17A-5 files for MAJESTIC SECURITIES LLC - CIK (1322861)
	Saved X-17A-5 files for MAJESTIC SECURITIES LLC year 07
7816 - Downloading X-17A-5 files for HAGGERTY & COMPANY, LLC - CIK (1323060)
7817 - Downloading X-17A-5 files for ALFANO, PATRICK MICHAEL - CIK (1330369)
7818 - Downloading X-17A-5 files for SHARPE 4 SECURITIES, LLC - CIK (1332098)
	Saved X-17A-5 files for SHARPE 4 SECURITIES, LLC year 06
7819 - Downloading X-17A-5 files for LANZARO, EDMOND SALVATORE - CIK (1348810)
7820 - Downloading X-17A-5 files for LABARBERA, JOSEPH ANTHONY - CIK (1353720)
7821 - Downloading X-17A-5 files for NEW SALEM INVESTMENT CAPITAL, LLC - CIK (1354968)
	Saved X-17A-5 files for NEW SALEM INVESTMENT CAPITAL, LLC year 07
7822 - Downloading X-17A-5 files for PRUTER, THOMAS WILLIAM - CIK (1362551)
7823 - Downloading X-17A-5 fi

	Saved X-17A-5 files for J.P.R. CAPITAL CORP. year 05
	Saved X-17A-5 files for J.P.R. CAPITAL CORP. year 04
	Saved X-17A-5 files for J.P.R. CAPITAL CORP. year 03
	Saved X-17A-5 files for J.P.R. CAPITAL CORP. year 02
7851 - Downloading X-17A-5 files for FIC INVESTMENT SERVICES, INC. - CIK (945040)
	Saved X-17A-5 files for FIC INVESTMENT SERVICES, INC. year 06
	Saved X-17A-5 files for FIC INVESTMENT SERVICES, INC. year 05
	Saved X-17A-5 files for FIC INVESTMENT SERVICES, INC. year 04
	Saved X-17A-5 files for FIC INVESTMENT SERVICES, INC. year 03
	Saved X-17A-5 files for FIC INVESTMENT SERVICES, INC. year 02
7852 - Downloading X-17A-5 files for LAXAGUE, ANTOINE GRATIEN - CIK (1022816)
7853 - Downloading X-17A-5 files for CITCO MUTUAL FUND DISTRIBUTORS, INC. - CIK (1023419)
	Saved X-17A-5 files for CITCO MUTUAL FUND DISTRIBUTORS, INC. year 06
	Saved X-17A-5 files for CITCO MUTUAL FUND DISTRIBUTORS, INC. year 05
	Saved X-17A-5 files for CITCO MUTUAL FUND DISTRIBUTORS, INC. year 04
	Saved X-

	Saved X-17A-5 files for BLUEBID BROKERAGE, LLC year 05
	Saved X-17A-5 files for BLUEBID BROKERAGE, LLC year 04
	Saved X-17A-5 files for BLUEBID BROKERAGE, LLC year 03
7879 - Downloading X-17A-5 files for GREENBOOK SECURITIES, INC. - CIK (1175777)
	Saved X-17A-5 files for GREENBOOK SECURITIES, INC. year 06
	Saved X-17A-5 files for GREENBOOK SECURITIES, INC. year 05
	Saved X-17A-5 files for GREENBOOK SECURITIES, INC. year 04
7880 - Downloading X-17A-5 files for SALEM DAVID - CIK (1186042)
7881 - Downloading X-17A-5 files for FIRST ADVANTAGE SECURITIES CORPORATION - CIK (1199897)
	Saved X-17A-5 files for FIRST ADVANTAGE SECURITIES CORPORATION year 06
	Saved X-17A-5 files for FIRST ADVANTAGE SECURITIES CORPORATION year 05
	Saved X-17A-5 files for FIRST ADVANTAGE SECURITIES CORPORATION year 04
7882 - Downloading X-17A-5 files for TALKING HORSE, LLC - CIK (1204934)
7883 - Downloading X-17A-5 files for AGS SPECIALISTS LLC - CIK (1208712)
7884 - Downloading X-17A-5 files for WALLACE, THOMAS M